<a href="https://colab.research.google.com/github/thilanC/PhysicianAI/blob/main/PhysicianAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Oct 16 18:57:18 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -qqq langchain
!pip install PyPDF2==2.12.1
!pip install -qqq sentence_transformers==2.2.2
!pip install -qqq InstructorEmbedding
!pip install -qqq pdf2image
!pip install pgvector psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.2 MB/s eta 0:00:00


In [ ]:
import torch
import os
import PyPDF2
from PyPDF2 import PdfFileReader
import pandas as pd
import numpy as np
import json
import psycopg2
import pgvector
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from langchain.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.text_splitter import RecursiveCharacterTextSplitter

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfFileReader(pdf_file)
        for page in range(pdf_reader.numPages):
            text += pdf_reader.getPage(page).extractText()
    return text

In [ ]:
# Directory where your folder structure is located
root_directory = "/content/drive/MyDrive/CDAZZDEV"

In [ ]:
# Create a RecursiveCharacterTextSplitter instance with specific parameters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len,)

In [ ]:
model = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE})

load INSTRUCTOR_Transformer
max_seq_length  512


In [ ]:
# Initialize lists to store data
data = []

# Iterate through the directory structure
for foldername, subfolders, filenames in os.walk(root_directory):
  for subfoldername in subfolders:
    for pdf_name in os.listdir(os.path.join(foldername, subfoldername)):
      print(pdf_name)
      if pdf_name.endswith(".pdf"):
        pdf_path = os.path.join(foldername, subfoldername, pdf_name)
        pdftext = extract_text_from_pdf(pdf_path)
        pdf_len = len(pdftext)
        if pdf_len<=1000:
          embedding = model.embed_query(pdftext)
          data.append({'foldername': foldername,'subfoldername': subfoldername,'pdf_name': pdf_name,'chunk_number':0,'pdftext': pdftext,'embeddings':embedding})
        else:
          split_text = text_splitter.split_text(pdftext)
          for i in range(len(split_text)):
            embedding = model.embed_query(split_text[i])
            data.append({'foldername': foldername,'subfoldername': subfoldername,'pdf_name': pdf_name,'chunk_number':i,'pdftext':split_text[i],'embeddings':embedding})

Glossopharyngeal neuralgia.pdf
Ear barotrauma.pdf
Patient perspective Managing pain during labor and delivery.pdf
Patient perspective Essential tremor with onset in childhood.pdf


In [ ]:
pdftext.count

<function str.count>

In [ ]:
# Create a DataFrame
df = pd.DataFrame(data)

# Add an 'id' column
df['id'] = range(1, len(df) + 1)

# Reorder the columns
df = df[['id', 'foldername', 'subfoldername', 'pdf_name', 'chunk_number','pdftext','embeddings']]

In [ ]:
# Display the DataFrame
print(df)


      id                       foldername        subfoldername  \
0      1  /content/drive/MyDrive/CDAZZDEV     Primary care ENT   
1      2  /content/drive/MyDrive/CDAZZDEV     Primary care ENT   
2      3  /content/drive/MyDrive/CDAZZDEV     Primary care ENT   
3      4  /content/drive/MyDrive/CDAZZDEV     Primary care ENT   
4      5  /content/drive/MyDrive/CDAZZDEV     Primary care ENT   
..   ...                              ...                  ...   
112  113  /content/drive/MyDrive/CDAZZDEV  Patient perspective   
113  114  /content/drive/MyDrive/CDAZZDEV  Patient perspective   
114  115  /content/drive/MyDrive/CDAZZDEV  Patient perspective   
115  116  /content/drive/MyDrive/CDAZZDEV  Patient perspective   
116  117  /content/drive/MyDrive/CDAZZDEV  Patient perspective   

                                              pdf_name  chunk_number  \
0                       Glossopharyngeal neuralgia.pdf             0   
1                       Glossopharyngeal neuralgia.pdf         

In [ ]:
df.head()

,id,foldername,subfoldername,pdf_name,chunk_number,pdftext,embeddings
0,1,/content/drive/MyDrive/CDAZZDEV,Primary care ENT,Glossopharyngeal neuralgia.pdf,0,"9/26/23, 9:25 PM Glossopharyngeal neuralgia - ...","[-0.009083596058189869, 0.005730495322495699, ..."
1,2,/content/drive/MyDrive/CDAZZDEV,Primary care ENT,Glossopharyngeal neuralgia.pdf,1,occurs significantly less frequently than othe...,"[-0.03840949013829231, 0.007060573901981115, -..."
2,3,/content/drive/MyDrive/CDAZZDEV,Primary care ENT,Glossopharyngeal neuralgia.pdf,2,"This topic last updated: Jun 14, 2023.\nSomati...","[-0.03875741362571716, -0.01355578564107418, -..."
3,4,/content/drive/MyDrive/CDAZZDEV,Primary care ENT,Glossopharyngeal neuralgia.pdf,3,dissections of the carotid sinus branch of the...,"[-0.048873744904994965, 0.010683140717446804, ..."
4,5,/content/drive/MyDrive/CDAZZDEV,Primary care ENT,Glossopharyngeal neuralgia.pdf,4,"baroreceptors (to monitor blood oxygen, carbon...","[-0.03425009921193123, -0.017318688333034515, ..."


In [ ]:
df.to_csv("/content/drive/MyDrive/CDAZZDEV/df.csv",escapechar='/')